In [ ]:
import rasterio
import numpy as np
import geopandas as gpd
from shapely.geometry import box

# Define the TIFF file path
tiff_file = 'h:\\Yehmh\\DNDF\\202404_DNDF\\DNDF7Band.tif'

# Open the TIFF image
with rasterio.open(tiff_file) as src:
    # Get metadata and calculate tile size in pixels
    meta = src.meta
    pixel_size = src.res[0]  # Assumes square pixels
    tile_size = 100  # Tile size in meters
    tile_size_pixels = int(tile_size / pixel_size)
    
    # Calculate the number of rows and columns for the grid
    rows = src.height // tile_size_pixels
    cols = src.width // tile_size_pixels
    
    # Prepare to store grid geometries and attributes
    grid = []

    # Loop through each tile in the grid
    for row in range(rows):
        for col in range(cols):
            # Define the window to read the tile
            window = rasterio.windows.Window(
                col * tile_size_pixels, row * tile_size_pixels, 
                tile_size_pixels, tile_size_pixels
            )

            # Read the last four bands (red, green, blue, infrared, red-edge)
            tile = src.read((1, 2, 3, 4, 5), window=window)
            
            # # Check if there are any pixels with value 0 or NaN
            # if np.any(tile == 0) or np.any(np.isnan(tile)):
            #     continue  # Skip this tile if condition is met

            # Calculate the proportion of pixels with value 0 or NaN
            zero_or_nan_ratio = np.mean((tile == 0) | np.isnan(tile), axis=(1, 2))

            # Skip the tile if more than 95% of the pixels are 0 or NaN in any band
            if np.any(zero_or_nan_ratio > 0.05):
                continue

            # Calculate squared deviations
            squared_deviation = (tile - tile.mean(axis=(1, 2), keepdims=True))**2
            
            # Calculate the sum of squares for each band
            sum_of_squares = np.sum(squared_deviation, axis=(1, 2))
            
            # Divide the sum by the number of pixels to get mean squared deviation
            mean_squared_deviation = sum_of_squares / (tile_size_pixels**2)
            
            # Calculate the total sum of squares across all bands
            total_sum_of_squares = np.sum(mean_squared_deviation)
            
            # Calculate FCSD (each band's mean squared deviation divided by total sum of squares)
            fcsd = mean_squared_deviation / total_sum_of_squares

            # Calculate NDVI using infrared (band 6) and red (band 4)
            nir = tile[3]  # Infrared band (band 6)
            red = tile[0]  # Red band (band 4)
            ndvi = (nir - red) / (nir + red)
            mean_ndvi = np.mean(ndvi)
            
            # Calculate the coordinates of the grid cell
            min_x, min_y = src.xy(row * tile_size_pixels, col * tile_size_pixels)
            max_x, max_y = src.xy((row + 1) * tile_size_pixels, (col + 1) * tile_size_pixels)
            
            # Create a polygon for the grid cell
            polygon = box(min_x, max_y, max_x, min_y)
            
            # Append the grid cell's data to the list
            grid.append({
                'geometry': polygon,
                'row': row,
                'col': col,
                'mean_squared_deviation_red': mean_squared_deviation[0],
                'mean_squared_deviation_green': mean_squared_deviation[1],
                'mean_squared_deviation_blue': mean_squared_deviation[2],
                'mean_squared_deviation_nir': mean_squared_deviation[3],
                'mean_squared_deviation_red_edge': mean_squared_deviation[4],
                'total_sum_of_squares': total_sum_of_squares,
                'fcsd_red': fcsd[0],
                'fcsd_green': fcsd[1],
                'fcsd_blue': fcsd[2],
                'fcsd_nir': fcsd[4],
                'fcsd_red_edge': fcsd[5],
                'mean_ndvi': mean_ndvi
            })

# Convert the list to a GeoDataFrame
gdf = gpd.GeoDataFrame(grid)

# Save the grid as a shapefile
gdf.to_file('h:\\Yehmh\\DNDF\\202404_DNDF\\spectral_diversity\\alpha_diversity_100_100_ndvi.shp')
